<a href="https://colab.research.google.com/github/arina19-2000/unimi/blob/main/tesi/tesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import pandas as pd

In [55]:
dfa = pd.read_csv('results adults.csv')

In [56]:
dfa.shape

(786, 273)

In [57]:
dfa.head(10)

,ID risposta,Data invio,Ultima pagina,Lingua iniziale,Seme,Data di inizio,Data dell'ultima azione,URL di riferimento,"In base a quanto previsto dal regolamento Ue n. 679/ 2016 e dal d. Lgs. 30 giugno 2003 n. 196 s.m.i. nonché dall'autorizzazione del garante della privacy n. 5/2002 al trattamento dei dati da parte di diverse categorie di titolari, i dati che la riguardano e che ci vengono liberamente comunicati, registrati su questo questionario online, sono protetti e anonimi. Il questionario, infatti, non chiede dati identificativi, non traccia l’IP, non memorizza i cookies dei rispondenti e in nessun modo è possibile risalire al compilatore. Le risposte al questionario saranno trattate in via tutto riservata e in forma aggregata da Coordinamento CARE per il tramite collaboratori specificatamente incaricati per le finalità connesse la propria attività di ricerca e non saranno comunicati a terzi se non in forma aggregata e anonima. Rispondere da parte sua è volontario e facoltativo e un eventuale rifiuto o interruzione non ha alcuna conseguenza. Il Titolare del trattamento dei dati è il Coordinamento CARE con sede legale in via Liberiana n.17 Roma (info@coordinamentocare.org) nella persona del legale rappresentante. La preghiamo quindi di manifestare il suo consenso scritto al trattamento da parte nostra dei dati anche classificati sensibili cliccando sulle caselle qui sotto la sua doppia autorizzazione quale requisito necessario per il proseguimento del questionario [LETTO (obbligatoria)]","In base a quanto previsto dal regolamento Ue n. 679/ 2016 e dal d. Lgs. 30 giugno 2003 n. 196 s.m.i. nonché dall'autorizzazione del garante della privacy n. 5/2002 al trattamento dei dati da parte di diverse categorie di titolari, i dati che la riguardano e che ci vengono liberamente comunicati, registrati su questo questionario online, sono protetti e anonimi. Il questionario, infatti, non chiede dati identificativi, non traccia l’IP, non memorizza i cookies dei rispondenti e in nessun modo è possibile risalire al compilatore. Le risposte al questionario saranno trattate in via tutto riservata e in forma aggregata da Coordinamento CARE per il tramite collaboratori specificatamente incaricati per le finalità connesse la propria attività di ricerca e non saranno comunicati a terzi se non in forma aggregata e anonima. Rispondere da parte sua è volontario e facoltativo e un eventuale rifiuto o interruzione non ha alcuna conseguenza. Il Titolare del trattamento dei dati è il Coordinamento CARE con sede legale in via Liberiana n.17 Roma (info@coordinamentocare.org) nella persona del legale rappresentante. La preghiamo quindi di manifestare il suo consenso scritto al trattamento da parte nostra dei dati anche classificati sensibili cliccando sulle caselle qui sotto la sua doppia autorizzazione quale requisito necessario per il proseguimento del questionario [MANIFESTO IL MIO CONSENSO (obbligatoria)]",...,Ha conseguito una laurea triennale o magistrale?.6,Voto della laurea? [Voto][.../110],Professione del PADRE quando il figlio ha fatto la scelta della scuola secondaria di II° grado o superiore:,Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [No],"Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alle primarie]","Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alla secondaria di primo grado]","Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alla secondaria di secondo grado]",Il PADRE ha dovuto ripetere degli anni alla scuola secondaria di secondo grado (o superiori)?,Il PADRE ha avuto difficoltà di rendimento/apprendimento che lo hanno spinto a cambiare tipologia di indirizzo di scuola secondaria di secondo grado (o superiore)?,Che tipologia di cambio?.10
0,1,2021-05-12 19:53:18,9.0,it,1.815587e+09,2021-05-12 16:59:58,2021-05-12 19:53:18,NaN,Sì,Sì,...,Laurea magistrale o vecchio ordinamento,110.0,programmista,No,No,No,Sì,"

## Data preparation

In [58]:
dfa = dfa.dropna(axis=1, how='all')
dfa.shape

(786, 191)

In [59]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('italian'))

def remove_italian_stopwords(col):
    col = col.lower()
    col = re.sub(r'[^\w\s]', '', col)
    # col = re.sub(r'^G\d+\[.*?\]\.\s*', '', col)
    col = re.sub(r'\s+', '_', col)
    words = [word for word in col.split('_') if word not in stop_words]
    return '_'.join(words)

dfa.columns = [remove_italian_stopwords(col) for col in dfa.columns]

dfa.columns




Index(['id_risposta', 'data_invio', 'ultima_pagina', 'lingua_iniziale', 'seme',
       'data_inizio', 'data_dellultima_azione', 'url_riferimento',
       'base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghia

In [60]:
dfa.shape

(786, 191)

In [48]:
column_types = dfa.dtypes

type_counts = column_types.value_counts()
print("Column types and counts:")
print(type_counts)

Column types and counts:
object     159
float64     31
int64        1
Name: count, dtype: int64


In [84]:
categorical_cols = dfa.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if col in dfa.columns:
        dfa[col] = dfa[col].fillna('No response')
    else:
        print(f"Column {col} does not exist in the DataFrame.")



In [85]:
dfa.head()

,id_risposta,data_invio,ultima_pagina,lingua_iniziale,seme,data_inizio,data_dellultima_azione,url_riferimento,base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_letto_obbligatoria,base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_manifesto_consenso_obbligatoria,...,conseguito_laurea_triennale_magistrale6,voto_laurea_voto110,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,tipologia_cambio10
0,1,2021-05-12 19:53:18,9.0,it,1.815587e+09,2021-05-12 16:59:58,2021-05-12 19:53:18,No response,Sì,Sì,...,Laurea magistrale o vecchio ordinamento,110.0,programmista,No,No,No,Sì,"Sì, 1 anno",Sì,da liceo scientifico a istituto professionale
1,2,2021-05-12 17:07:15,9.0,it,5.185927e+07,2021-05-12 17:06:59,2021-05-12 17:07:15,No response,Sì,Sì,...,No response,NaN,No response,No response,No response,No response,No response,No response,No response,No response
2,3,2021-05-12 18:33:39,9.0,it,4.972766e+08,2021-05-12 17:15:01,2021-05-12 18:33:39,No response,Sì,Sì,...,No response,NaN,Operaio,Sì,No,No,No,No response,No response,No response
3,4,2021-05-12 17:31:35,9.0,it,8.163335e+07,2021-05-12 17:25:31,2021-05-12 17:31:35,No response,Sì,Sì,...,Laurea magistrale o vecchio ordinamento,110.0,Impiegato,Sì,No,No,No,No,No,No response
4,5,2021-05-12 21:39:05,9.0,it,8.177464e+08,2021-05-12 18:11:54,2021-05-12 21:39:05,No response,Sì,No,...,No response,110.0,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No,No response


##Children survey results

In [45]:
dfc = pd.read_excel('results kids.xlsx')

In [37]:
dfc.shape

(235, 93)

In [46]:
dfc = dfc.dropna(axis=1, how='all')
dfc.shape

(235, 91)

In [47]:
dfc.columns = [remove_italian_stopwords(col) for col in dfc.columns]

dfc.columns

Index(['id_id_risposta', 'submitdate_data_invio', 'lastpage_ultima_pagina',
       'startlanguage_lingua_iniziale', 'seed_seme', 'startdate_data_inizio',
       'datestamp_data_dellultima_azione', 'refurl_url_riferimento',
       'privacysq001_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_leg

In [49]:
column_types = dfc.dtypes
type_counts = column_types.value_counts()

print("Column types and counts:")
print(type_counts)

Column types and counts:
object            72
float64           14
datetime64[ns]     3
int64              2
Name: count, dtype: int64


In [87]:
categorical_cols = dfc.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if col in dfc.columns:
        dfc[col] = dfc[col].fillna('No response')
    else:
        print(f"Column {col} does not exist in the DataFrame.")

In [88]:
dfc.head()

,id_id_risposta,submitdate_data_invio,lastpage_ultima_pagina,startlanguage_lingua_iniziale,seed_seme,startdate_data_inizio,datestamp_data_dellultima_azione,refurl_url_riferimento,privacysq001_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_letto_obbligatoria,privacysq002_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_manifesto_consenso_obbligatoria,...,g8q00003a_conseguito_laurea_triennale_magistrale,g8q00004sq001_sq001_voto_laurea_voto110,g8q00005_professione_padre_quando_fatto_scelta_scuola_secondaria_ii_grado_superiore,g8q00006sq001_padre_mai_stato_organi_collegiali_scuole_propri_figli_no,g8q00006sq002_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,g8q00006sq005_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,g8q00006sq004_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,g8q00007_padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,g8q00008_padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,g8q00009_tipologia_cambio
0,1,NaT,0.0,it,433164667,2021-07-03 08:57:45,2021-07-03 08:57:51,No response,No,No,...,No response,NaN,No response,No response,No response,No response,No response,No response,No response,No response
1,2,2021-07-06 16:52:01,5.0,it,696014843,2021-07-06 16:48:47,2021-07-06 16:52:01,http://m.facebook.com/,Sì,Sì,...,No response,NaN,Impiegato,Sì,No,No,No,No response,No response,No response
2,3,2021-07-06 16:55:44,5.0,it,531131002,2021-07-06 16:49:03,2021-07-06 16:55:44,No response,Sì,Sì,...,No response,110.0,Docente universitario,Sì,No,No,No,No,No,No response
3,4,NaT,0.0,it,1471963680,2021-07-06 16:49:26,2021-07-06 16:49:39,http://m.facebook.com/,No,No,...,No response,NaN,No response,No response,No response,No response,No response,No response,No response,No response
4,5,NaT,2.0,it,1652562375,2021-07-06 16:54:19,2021-07-06 16:54:47,https://m.facebook.com/,Sì,Sì,...,No response,NaN,No response,No response,No response,No response,No 